# Lesson 1 - Semantic Search

### Import the Needed Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

from DLAIUtils import Utils
import DLAIUtils

import os
import time
import torch

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
from tqdm.auto import tqdm

### Load the Dataset

In [5]:
dataset = load_dataset('quora', split='train[240000:290000]')

Using the latest cached version of the module from /home/mark/.cache/huggingface/modules/datasets_modules/datasets/quora/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04 (last modified on Mon Feb 26 22:16:14 2024) since it couldn't be found locally at quora., or remotely on the Hugging Face Hub.
Found cached dataset quora (/home/mark/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


In [6]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [7]:
# Extract questions and drop ids
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
# Remove duplicate questions
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')
print(f'Number of unique questions: {len(question)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000
Number of unique questions: 88919


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [9]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

(384,)

### Setup Pinecone

In [10]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()
PINECONE_API_KEY = "937d27db-fa5f-4b61-a6b7-2b756f4ef223"
pinecone = Pinecone(api_key=PINECONE_API_KEY)
print(pinecone.list_indexes())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/urllib3/connection.py:174 in        │
│ _new_conn                                                                                        │
│                                                                                                  │
│   171 │   │   │   extra_kw["socket_options"] = self.socket_options                               │
│   172 │   │                                                                                      │
│   173 │   │   try:                                                                               │
│ ❱ 174 │   │   │   conn = connection.create_connection(                                           │
│   175 │   │   │   │   (self._dns_host, self.port), self.timeout, **extra_kw                      │
│   176 │   │   │   )                                                                              │
│   177                                                                                            │
│                                                                                                  │
│ /home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/urllib3/util/connection.py:72 in    │
│ create_connection                                                                                │
│                                                                                                  │
│    69 │   │   │   LocationParseError(u"'%s', label empty or too long" % host), None              │
│    70 │   │   )                                                                                  │
│    71 │                                                                                          │
│ ❱  72 │   for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):                 │
│    73 │   │   af, socktype, proto, canonname, sa = res                                           │
│    74 │   │   sock = None                                                                        │
│    75 │   │   try:                                                                               │
│                                                                                                  │
│ /home/mark/anaconda3/envs/OpenAI/lib/python3.9/socket.py:954 in getaddrinfo                      │
│                                                                                                  │
│   951 │   # We override this function since we want to translate the numeric family              │
│   952 │   # and socket type values to enum constants.                                            │
│   953 │   addrlist = []                                                                          │
│ ❱ 954 │   for res in _socket.getaddrinfo(host, port, family, type, proto, flags):                │
│   955 │   │   af, socktype, proto, canonname, sa = res                                           │
│   956 │   │   addrlist.append((_intenum_converter(af, AddressFamily),                            │
│   957 │   │   │   │   │   │    _intenum_converter(socktype, SocketKind),                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/urllib3/connectionpool.py:715 in    │
│ urlopen                                                                                          │
│                                                                                                  │
│    712 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    713 │   │   │                           

In [ ]:
#INDEX_NAME = utils.create_dlai_index_name('dl-ai')
INDEX_NAME="openai-labs-384"

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
pinecone.create_index(name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-west-2'))

index = pinecone.Index(INDEX_NAME)
print(index)

### Create Embeddings and Upsert to Pinecone

In [ ]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

range(0, len(questions), batch_size)

i = 0
i_end = 200
ids = [str(x) for x in range(i, i_end)]
print(ids)
metadatas = [{'text': text} for text in questions[i:i_end]]
print(metadatas)
xc = model.encode(questions[i:i_end])
print(xc)

In [ ]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

In [ ]:
index.describe_index_stats()

### Run Your Query

In [ ]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [ ]:
run_query('which city has the highest population in the world?')

In [ ]:
query = 'how do i make chocolate cake?'
run_query(query)